In [30]:
import sys
sys.path.append('..')

from ml.dataset.corpus import *
from ml.dataset.mappers_preprocess import *
from ml.dataset.mappers_simplify import *
from ml.dataset.pipeline import *
from IPython.core.display import HTML, display

import matplotlib.pyplot as plt
import pandas as pd
import pychord
import music21
music21.environment.set('musescoreDirectPNGPath', '/usr/bin/musescore')

import matplotlib.pyplot as plt
import numpy as np
import pickle

y = np.load("y_simple.npy")
X = (np.load("X_simple.npy"))

In [33]:
print(X.shape, y.shape)

(30009, 56) (30009,)


In [32]:
els, counts = np.unique(y, return_counts=True)
# plt.scatter(range(len(els)), counts)
# plt.show()
# print(els, counts)
rare = els[counts < 10]

isin = np.logical_not(np.isin(y, rare))
print(X.shape)
X = X[isin, :]
print(X.shape)
y = y[isin]

els, counts = np.unique(y, return_counts=True)
# plt.scatter(range(len(els)), counts)
# plt.show()
print(els)
print(len(els))

feat = X.shape[1]
print("Features:", feat)
tst = np.array([1])
print(tst.dtype, X.dtype)
if X.dtype == tst.dtype:
    num_features = np.zeros(feat, dtype='bool')
else:
    num_features = np.array(['0' <= X[0, i][0] <= '9' for i in range(feat)])
cat_features = np.logical_not(num_features)
print(num_features)
print(cat_features)

(30029, 56)
(30009, 56)
['' 'A1' 'A2' 'A3' 'A4' 'A5' 'B1' 'B2' 'B3' 'B4' 'Bb1' 'Bb2' 'Bb3' 'Bb4'
 'Bb5' 'C#2' 'C#3' 'C#4' 'C#5' 'C2' 'C3' 'C4' 'C5' 'C6' 'D2' 'D3' 'D4'
 'D5' 'E1' 'E2' 'E3' 'E4' 'E5' 'Eb2' 'Eb3' 'Eb4' 'Eb5' 'F#2' 'F#3' 'F#4'
 'F#5' 'F1' 'F2' 'F3' 'F4' 'F5' 'F6' 'G#2' 'G#3' 'G#4' 'G2' 'G3' 'G4' 'G5']
54
Features: 56
int64 <U32
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False]
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  Tr

In [34]:
# X_cat = X[:,8:16]
# X_real = np.int_(np.hstack([X[:,:8], X[:,16:]]))

X_cat = X[:, cat_features]
X_real = np.int_(X[:, num_features])
print(X_cat[0, :])
print(X_real[0, :])

['Bb2' 'Bb2' 'Bb2' 'Bb2' 'Bb2' 'Bb2' 'C3' 'C3' 'C3' 'C3' 'C3' 'C3' 'A2'
 'A2' 'A2' 'A2' 'A2' 'A2' 'Bb2' 'Bb2' 'Bb2' 'Bb2' 'Bb2' 'Bb2' 'G2' 'G2'
 'G2' 'G2']
[34 34 34 34 34 34 36 36 36 36 36 36 33 33 33 33 33 33 34 34 34 34 34 34
 31 31 31 31]


In [35]:
X_cat

array([['Bb2', 'Bb2', 'Bb2', ..., 'G2', 'G2', 'G2'],
       ['C3', 'C3', 'C3', ..., 'C3', 'C3', 'C3'],
       ['A2', 'A2', 'Bb2', ..., 'Eb3', 'F2', 'F2'],
       ...,
       ['D2', 'D2', 'D2', ..., 'G2', 'G2', 'G2'],
       ['D2', 'D2', 'D2', ..., 'G2', 'G2', 'G2'],
       ['D2', 'D2', 'D2', ..., 'G2', 'G2', 'G2']], dtype='<U32')

In [36]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(X_cat.reshape(X_cat.size))
for i in range(sum(cat_features)):
    print(enc.classes_)
for i in range(sum(cat_features)):
    X_cat[:, i] = enc.transform(X_cat[:, i])

['' 'A1' 'A2' 'A3' 'A4' 'A5' 'A6' 'B1' 'B2' 'B3' 'B4' 'B5' 'Bb1' 'Bb2'
 'Bb3' 'Bb4' 'Bb5' 'C#2' 'C#3' 'C#4' 'C#5' 'C2' 'C3' 'C4' 'C5' 'C6' 'D2'
 'D3' 'D4' 'D5' 'D6' 'E1' 'E2' 'E3' 'E4' 'E5' 'E6' 'Eb2' 'Eb3' 'Eb4' 'Eb5'
 'Eb6' 'F#2' 'F#3' 'F#4' 'F#5' 'F#6' 'F1' 'F2' 'F3' 'F4' 'F5' 'F6' 'G#2'
 'G#3' 'G#4' 'G#5' 'G1' 'G2' 'G3' 'G4' 'G5' 'G6']
['' 'A1' 'A2' 'A3' 'A4' 'A5' 'A6' 'B1' 'B2' 'B3' 'B4' 'B5' 'Bb1' 'Bb2'
 'Bb3' 'Bb4' 'Bb5' 'C#2' 'C#3' 'C#4' 'C#5' 'C2' 'C3' 'C4' 'C5' 'C6' 'D2'
 'D3' 'D4' 'D5' 'D6' 'E1' 'E2' 'E3' 'E4' 'E5' 'E6' 'Eb2' 'Eb3' 'Eb4' 'Eb5'
 'Eb6' 'F#2' 'F#3' 'F#4' 'F#5' 'F#6' 'F1' 'F2' 'F3' 'F4' 'F5' 'F6' 'G#2'
 'G#3' 'G#4' 'G#5' 'G1' 'G2' 'G3' 'G4' 'G5' 'G6']
['' 'A1' 'A2' 'A3' 'A4' 'A5' 'A6' 'B1' 'B2' 'B3' 'B4' 'B5' 'Bb1' 'Bb2'
 'Bb3' 'Bb4' 'Bb5' 'C#2' 'C#3' 'C#4' 'C#5' 'C2' 'C3' 'C4' 'C5' 'C6' 'D2'
 'D3' 'D4' 'D5' 'D6' 'E1' 'E2' 'E3' 'E4' 'E5' 'E6' 'Eb2' 'Eb3' 'Eb4' 'Eb5'
 'Eb6' 'F#2' 'F#3' 'F#4' 'F#5' 'F#6' 'F1' 'F2' 'F3' 'F4' 'F5' 'F6' 'G#2'
 'G#3' 'G#4' 'G#5' 'G1' 

In [37]:
X_cat

array([['13', '13', '13', ..., '58', '58', '58'],
       ['22', '22', '22', ..., '22', '22', '22'],
       ['2', '2', '13', ..., '38', '48', '48'],
       ...,
       ['26', '26', '26', ..., '58', '58', '58'],
       ['26', '26', '26', ..., '58', '58', '58'],
       ['26', '26', '26', ..., '58', '58', '58']], dtype='<U32')

In [38]:
X = np.hstack([np.int_(X_cat), X_real])

In [39]:
X

array([[13, 13, 13, ..., 31, 31, 31],
       [22, 22, 22, ..., 36, 36, 36],
       [ 2,  2, 13, ..., 39, 41, 41],
       ...,
       [26, 26, 26, ..., 45, 45, 45],
       [26, 26, 26, ..., 45, 45, 45],
       [26, 26, 26, ..., 45, 45, 45]])

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from nottingham_test import custom_scorers
from sklearn.metrics import make_scorer

clf = RandomForestClassifier(n_estimators=20)
clf.fit(X_train, y_train)

print(X_test)

#abs_dist_score = make_scorer(custom_scorers.chords_dist_error)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
#print(abs_dist_score(clf, X_test, y_test))

with open('rf_nottingham.pkl', 'wb') as fid:
    pickle.dump(clf, fid)
print(y_test[10], X_test[10, :], clf.predict(X_test)[10])

[[ 8  8  8 ... 54 54 54]
 [ 3  0  3 ... 50 50 50]
 [27 27 32 ... 33 38 38]
 ...
 [24 24 24 ... 33 33 33]
 [ 0  0 22 ...  0  0  0]
 [32 32 32 ... 59 57  0]]
0.8006130880980941
F4 [ 0 50  0 50  0 50  0 50  0 49  0 49  0 49  0 49  0 15  0 15  0 15  0 15
  0 24  0 24 49 49 49 49 49 49 49 49 41 41 41 41 41 41 41 41 42 42 42 42
 42 42 42 42 44 44 44 44] F4
